This notebook shows how to generate descriptors with few examples.

In [1]:
import numpy as np
from asymopt.desc import StericDescriptor

## Generate PhysOrg descriptors

The electronic descriptors were generated through single-point energy calculations using xTB, and they were extracted and saved in the directory like: './ind_3d_geom/xx/xtb/xx/desc_ens.npy'.

### Ligand (ligand-metal complex)

In [2]:
# use 'xTBCalculator' and 'xTBDescriptor' class can generate follow meta files:
eg_init_file = '../demo/ind_3d_geom/lig_0/xtb/lig_0/lig_0.xyz'
eg_opted_file = '../demo/ind_3d_geom/lig_0/xtb/lig_0/xtbopt.xyz'
eg_elec_desc_file = '../demo/ind_3d_geom/lig_0/xtb/lig_0/desc_ens.npy'
with open(eg_init_file,'r') as fr:
    lines = fr.readlines()
metal = lines[1].strip().split('///')[1]
coords = lines[2:]
symbols = [coord.strip().split()[0] for coord in coords]
metal_idx = [idx for idx in range(len(symbols)) if symbols[idx] == metal][0]
desc_ens = np.load(eg_elec_desc_file,allow_pickle=True).item()
xtb_global_desc = np.concatenate([[val] if isinstance(val,float) else val for val in list(desc_ens['global'].values())])
xtb_desc_metal = np.concatenate([desc_ens['local'][key][metal_idx] for key in ['charges','fukui']])

stereo_desc_calc = StericDescriptor(eg_opted_file)
stero_desc = np.array(stereo_desc_calc.BV(metal_idx))
lig_physorg_desc = np.concatenate([xtb_global_desc,xtb_desc_metal,stero_desc])

### Reactant 1

In [5]:
eg_init_file = '../demo/ind_3d_geom/rct_1_0/xtb/rct_1_0/rct_1_0.xyz'
eg_opted_file = '../demo/ind_3d_geom/rct_1_0/xtb/rct_1_0/xtbopt.xyz'
eg_elec_desc_file = '../demo/ind_3d_geom/rct_1_0/xtb/rct_1_0/desc_ens.npy'

with open(eg_init_file,'r') as fr:
    lines = fr.readlines()
attach_idx_lst = eval(lines[1].strip().split('///')[1])
group_idx_lst = eval(lines[1].strip().split('///')[2])

desc_ens = np.load(eg_elec_desc_file,allow_pickle=True).item()
xtb_global_desc = np.concatenate([[val] if isinstance(val,float) else val for val in list(desc_ens['global'].values())])
attach_idx = attach_idx_lst[0]
xtb_desc_attach = np.concatenate([desc_ens['local'][key][attach_idx] for key in ['charges','fukui']])
stereo_desc_calc = StericDescriptor(eg_opted_file)
stero_desc = np.array(stereo_desc_calc.BV(attach_idx) + stereo_desc_calc.Sterimol(attach_idx,group_idx_lst[0][0]))
rct1_physorg_desc = np.concatenate([xtb_global_desc,xtb_desc_attach,stero_desc])

### Reactant 2

In [6]:
eg_init_file = '../demo/ind_3d_geom/rct_2_0/xtb/rct_2_0/rct_2_0.xyz'
eg_opted_file = '../demo/ind_3d_geom/rct_2_0/xtb/rct_2_0/xtbopt.xyz'
eg_elec_desc_file = '../demo/ind_3d_geom/rct_2_0/xtb/rct_2_0/desc_ens.npy'

with open(eg_init_file,'r') as fr:
    lines = fr.readlines()
coords = lines[2:]
symbols = [coord.strip().split()[0] for coord in coords]

attach_idx_lst = eval(lines[1].strip().split('///')[1])
group_idx_lst = eval(lines[1].strip().split('///')[2])

desc_ens = np.load(eg_elec_desc_file,allow_pickle=True).item()
xtb_global_desc = np.concatenate([[val] if isinstance(val,float) else val for val in list(desc_ens['global'].values())])
attach_idx = attach_idx_lst[0]
B_idx = [idx for idx in group_idx_lst[0] if symbols[idx] == 'B'][0]

xtb_desc_attach = np.concatenate([desc_ens['local'][key][attach_idx] for key in ['charges','fukui']])

stereo_desc_calc = StericDescriptor(eg_opted_file)
stero_desc = np.array(stereo_desc_calc.BV(attach_idx) + stereo_desc_calc.Sterimol(attach_idx,B_idx))

rct2_physorg_desc = np.concatenate([xtb_global_desc,xtb_desc_attach,stero_desc])

## Other 3D-geometry-based descrptor

In [1]:
from dscribe.descriptors import MBTR,ACSF
from ase.io import read as ase_read
from asymopt.geom import xyz2coords,find_closest_to_center

In [2]:
r1_species = ['O', 'P', 'F', 'Br', 'N', 'I',
              'C', 'S', 'H', 'Si', 'Cl']
r2_species = ['O', 'F', 'K', 'B', 'C', 'H']
lig_species = ['O', 'P', 'F', 'Pd', 'N', 'Fe',
               'C', 'S', 'Si', 'Cl', 'Ni', 'H']  ## metal-ligand complex

### ACSF

In [3]:
r1_acsf = ACSF(
    species=r1_species,
    r_cut=6.0,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]],
)
r2_acsf = ACSF(
    species=r2_species,
    r_cut=6.0,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]],
)
lig_acsf = ACSF(
    species=lig_species,
    r_cut=6.0,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]],
) ## metal-ligand complex

In [4]:
init_rct1_xyz_file = '../demo/ind_3d_geom/rct_1_0/xtb/rct_1_0/rct_1_0.xyz'
init_rct2_xyz_file = '../demo/ind_3d_geom/rct_2_0/xtb/rct_2_0/rct_2_0.xyz'
init_lig_xyz_file = '../demo/ind_3d_geom/lig_0/xtb/lig_0/lig_0.xyz'

with open(init_rct1_xyz_file, 'r') as fr:
    lines = fr.readlines()
rct1_smi = lines[1].strip().split('///')[0]
rct1_opt_xyz_file = '/'.join(init_rct1_xyz_file.split('/')[:-1]+['xtbopt.xyz'])
rct1_coords = xyz2coords(rct1_opt_xyz_file)
rct1_key_idx = find_closest_to_center(rct1_coords)
rct1_ase_atoms = ase_read(rct1_opt_xyz_file,format='xyz')
rct1_desc = r1_acsf.create(rct1_ase_atoms,centers=[rct1_key_idx])
rct1_desc = rct1_desc.reshape(-1)

with open(init_rct2_xyz_file, 'r') as fr:
    lines = fr.readlines()
rct2_smi = lines[1].strip().split('///')[0]
rct2_opt_xyz_file = '/'.join(init_rct2_xyz_file.split('/')[:-1]+['xtbopt.xyz'])
rct2_coords = xyz2coords(rct2_opt_xyz_file)
rct2_key_idx = find_closest_to_center(rct2_coords)
rct2_ase_atoms = ase_read(rct2_opt_xyz_file,format='xyz')
rct2_desc = r2_acsf.create(rct2_ase_atoms,centers=[rct2_key_idx])
rct2_desc = rct2_desc.reshape(-1)

with open(init_lig_xyz_file, 'r') as fr:
    lines = fr.readlines()
lig_smi,metal = lines[1].strip().split('///')[:2]
lig_opt_xyz_file = '/'.join(init_lig_xyz_file.split('/')[:-1]+['xtbopt.xyz'])
lig_coords = xyz2coords(lig_opt_xyz_file)
lig_key_idx = find_closest_to_center(lig_coords)
lig_ase_atoms = ase_read(lig_opt_xyz_file,format='xyz')
lig_desc = lig_acsf.create(lig_ase_atoms,centers=[lig_key_idx])
lig_desc = lig_desc.reshape(-1)

### MBTR

In [5]:
r1_mbtr = MBTR(
    species=r1_species,
    geometry={"function": "inverse_distance"},
    grid={"min": 0, "max": 1, "n": 100, "sigma": 0.1},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=False,
    normalization="l2",
)

r2_mbtr = MBTR(
    species=r2_species,
    geometry={"function": "inverse_distance"},
    grid={"min": 0, "max": 1, "n": 100, "sigma": 0.1},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=False,
    normalization="l2",
)

lig_mbtr = MBTR(
    species=lig_species,
    geometry={"function": "inverse_distance"},
    grid={"min": 0, "max": 1, "n": 100, "sigma": 0.1},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=False,
    normalization="l2",
)

In [6]:
r1_mbtr = MBTR(
    species=r1_species,
    geometry={"function": "inverse_distance"},
    grid={"min": 0, "max": 1, "n": 100, "sigma": 0.1},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=False,
    normalization="l2",
)

r2_mbtr = MBTR(
    species=r2_species,
    geometry={"function": "inverse_distance"},
    grid={"min": 0, "max": 1, "n": 100, "sigma": 0.1},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=False,
    normalization="l2",
)

lig_mbtr = MBTR(
    species=lig_species,
    geometry={"function": "inverse_distance"},
    grid={"min": 0, "max": 1, "n": 100, "sigma": 0.1},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=False,
    normalization="l2",
)